# Importation of libraries

In [ ]:
# Standard libraries
import pandas as pd
import time
import pickle

# B. Loading of datasets

## 1. Revenues

In [ ]:
df_revenues = pd.read_excel(r"Database\revenues_pseudonymized.xlsx")

In [ ]:
df_revenues

## 2. Credit Sales (EDA)

In [ ]:
from FeatureEngineering._ARCHIVED_credit_sales_eda_single import CreditSales

start = time.time()
cs = CreditSales(df_revenues)
end = time.time()

print(f"Execution time: {end - start:.6f} seconds")

df_truth = cs.show_data()
df_truth = df_truth.groupby(['school_year', 'student_id_pseudonimized', 'category_name']).sum(numeric_only=True)

from FeatureEngineering._ARCHIVED_credit_sales_eda_multiple_brute import CreditSales as CreditSalesOptimized

start = time.time()
cs_brute = CreditSalesOptimized(df_revenues)
end = time.time()

print(f"Execution time: {end - start:.6f} seconds")

# Save (pickle) the variable
with open("cs_brute.pkl", "wb") as f:
    pickle.dump(cs_brute, f)

print("cs_brute has been saved to cs_brute.pkl")

In [ ]:
# Later on, you can load (unpickle) it back:
with open("cs_brute.pkl", "rb") as f:
    cs_brute = pickle.load(f)

print("Restored:", cs_brute)

In [ ]:
from FeatureEngineering.credit_sales_eda import CreditSales as CreditSalesOptimized

start = time.time()
cs_opti = CreditSalesOptimized(df_revenues)
end = time.time()

print(f"Execution time: {end - start:.6f} seconds")

In [ ]:
df_brute = cs_brute.show_data()
df_brute.reset_index().to_excel("credit_sales_brute.xlsx", index=False)

In [ ]:
df_test_opti = cs_opti.show_data()
df_test_opti.to_excel("credit_sales_test.xlsx", index=False)

In [ ]:
print("rows")
print(f"Truth : {df_truth.reset_index().shape[0]}")
print(f"Brute : {df_brute.reset_index().shape[0]}")
print(f"Test  : {df_test_opti.reset_index().shape[0]}")

In [ ]:
import pandas as pd

# Merge with indicator to see where rows come from
merged = df_test_opti.reset_index().merge(
    df_brute.reset_index(),
    on=["school_year", "student_id_pseudonimized", "category_name"],
    how="left",
    indicator=True
)

# Rows that do NOT exist in brute
missing_in_brute = merged[merged["_merge"] == "left_only"]

missing_in_brute[["school_year", "student_id_pseudonimized", "category_name"]]

In [ ]:
import pandas as pd

# Merge the two DataFrames on the keys
merged = df_truth.merge(
    df_test_opti,
    on=["school_year", "student_id_pseudonimized", "category_name"],
    suffixes=("_truth", "_multiple")
)

# Find rows where net_receivables differ
diff_rows = merged[merged["net_receivables_truth"] != merged["net_receivables_multiple"]]

diff_rows[['net_receivables_truth', 'net_receivables_multiple']] 

In [ ]:
df_brute = df_brute.groupby(['school_year', 'student_id_pseudonimized', 'category_name']).sum(numeric_only=True)
df_test_opti = df_test_opti.groupby(['school_year', 'student_id_pseudonimized', 'category_name']).sum(numeric_only=True)

In [ ]:
print("discounts")
print(f"Truth : {df_truth.amount_discounted.sum()}")
print(f"Brute : {df_brute.amount_discounted.sum()}")
print(f"Test  : {df_test_opti.amount_discounted.sum()}")

In [ ]:
print("adjustments")
print(f"Truth : {df_truth.adjustments.sum()}")
print(f"Brute : {df_brute.adjustments.sum()}")
print(f"Test  : {df_test_opti.adjustments.sum()}")

In [ ]:
print("payments")
print(f"Truth : {df_truth.total_payments.sum()}")
print(f"Brute : {df_brute.total_payments.sum()}")
print(f"Test  : {df_test_opti.total_payments.sum()}")

In [ ]:
print("net receivables")
print(f"Truth : {df_truth.net_receivables.sum()}")
print(f"Brute : {df_brute.net_receivables.sum()}")
print(f"Test  : {df_test_opti.net_receivables.sum()}")

In [ ]:
df_truth.reset_index()

In [ ]:
df_brute.reset_index()

In [ ]:
df_test_opti.reset_index()

## 2. Credit Sales (Machine learning)

In [ ]:
from FeatureEngineering.credit_sales_eda import CreditSales as CreditSalesEDA

cs_eda = CreditSalesEDA(df_revenues)
df_cs_fact = cs_eda.show_data()

In [ ]:
from FeatureEngineering.credit_sales_machine_learning import CreditSales

cs = CreditSales(df_revenues, df_enrollees)
df_credit_sales = cs.show_data()

In [ ]:
fact = df_cs_fact[df_cs_fact['total_payments'] != 0].count()[0]
test = df_credit_sales[df_credit_sales['date_fully_paid'].notna()].count()[0]


print(fact)
print(test)
print(fact-test)

In [ ]:
print(len(df_cs_fact))
print(len(df_credit_sales))

In [ ]:
import pandas as pd

# Step 1: Boolean flags
df_fact_flag = df_cs_fact.assign(
    has_payment = df_cs_fact['total_payments'] != 0
)

df_sales_flag = df_credit_sales.assign(
    has_payment = df_credit_sales['date_fully_paid'].notna()
)

# Step 2: Merge with sales as the left table
merged = df_sales_flag.merge(
    df_fact_flag,
    on=['school_year', 'student_id_pseudonimized', 'category_name'],
    how='right',
    suffixes=('_sales', '_fact')
)

# Step 3: Identify entries missing in fact
missing_in_fact = merged[
    (merged['has_payment_sales']) & 
    (merged['has_payment_fact'].isna())
]

# Step 4: Counts
fact_count = df_fact_flag['has_payment'].sum()
sales_count = df_sales_flag['has_payment'].sum()
missing_count = len(missing_in_fact)

print("Fact count:", fact_count)
print("Sales count:", sales_count)
print("Missing in fact:", missing_count)

In [ ]:
import pandas as pd

# Step 1: Boolean flags
df_fact_flag = df_cs_fact.assign(
    has_payment = df_cs_fact['total_payments'] != 0
)

df_sales_flag = df_credit_sales.assign(
    has_payment = df_credit_sales['date_fully_paid'].notna()
)

# Step 2: Merge with fact as the left table (since we want to check missing in sales)
merged = df_fact_flag.merge(
    df_sales_flag,
    on=['school_year', 'student_id_pseudonimized', 'category_name'],
    how='left',
    suffixes=('_fact', '_sales')
)

# Step 3: Identify entries missing in sales
missing_in_sales = merged[
    (merged['has_payment_fact']) & 
    (merged['has_payment_sales'].isna())
]

# Step 4: Counts
fact_count = df_fact_flag['has_payment'].sum()
sales_count = df_sales_flag['has_payment'].sum()
missing_count = len(missing_in_sales)

print("Fact count:", fact_count)
print("Sales count:", sales_count)
print("Missing in sales:", missing_count)

In [ ]:
# Define the composite keys
keys = ['school_year', 'student_id_pseudonimized', 'category_name', 'due_date']

# Filter the two DataFrames according to your conditions
fact_filtered = df_cs_fact[df_cs_fact['total_payments'] != 0]
test_filtered = df_credit_sales[df_credit_sales['date_fully_paid'].notna()]

# Merge with indicator to see differences
diff_rows = fact_filtered.merge(
    test_filtered,
    on=keys,
    how='outer',
    indicator=True
)

# Extract rows that are different (exist only in one side)
diff_only = diff_rows[diff_rows['_merge'] != 'both']

diff_only

In [ ]:
print(df_cs_fact.credit_sale_amount.sum())
print(df_credit_sales.credit_sale_amount.sum())